# Algoritmos SOTA Avanzados en Deep Reinforcement Learning

## Guía Completa: PPO, DDPG, TD3 y SAC

Este notebook explora los algoritmos de Reinforcement Learning más modernos y efectivos, preparándote para meta-learning y aplicaciones avanzadas.

**Autores**: MARK-126  
**Fecha**: 2024  
**Nivel**: Avanzado

## 1. Introducción a Algoritmos SOTA (3 celdas)

### 1.1 ¿Qué son los algoritmos SOTA (State-of-the-Art)?

Los algoritmos SOTA representan la frontera actual del conocimiento en Deep RL. Estos algoritmos:

1. **PPO (Proximal Policy Optimization)**
   - Algoritmo on-policy basado en policy gradient
   - Usa clipping para asegurar updates seguros
   - Versátil: funciona en espacios discretos y continuos
   - Usado en: OpenAI Five, ChatGPT training (RLHF)

2. **DDPG (Deep Deterministic Policy Gradient)**
   - Primer algoritmo actor-critic para control continuo
   - Política determinista + ruido para exploración
   - Off-policy: sample efficient
   - Base para TD3 y SAC

3. **TD3 (Twin Delayed DDPG)**
   - Mejora significativa sobre DDPG
   - Twin critics reducen sobreestimación
   - Delayed policy updates reducen varianza
   - Target policy smoothing para robustez

4. **SAC (Soft Actor-Critic)**
   - Maximum entropy RL framework
   - Auto-tuning de temperatura
   - Stochastic policy más exploratoria
   - SOTA actual para control continuo

**Clasificación**:
- **On-policy vs Off-policy**: PPO es on-policy; DDPG, TD3, SAC son off-policy
- **Deterministic vs Stochastic**: DDPG y TD3 son deterministas; PPO y SAC son estocásticos
- **Discrete vs Continuous**: PPO soporta ambas; DDPG, TD3, SAC solo continuas

### 1.2 Comparación Conceptual

| Algoritmo | Tipo | Acción | Estabilidad | Exploración | Velocidad | Uso Ideal |
|-----------|------|--------|-------------|-------------|-----------|----------|
| PPO | On-Policy | Ambas | Media | Entropy | Rápida | Robotics, RL-Training |
| DDPG | Off-Policy | Continua | Baja | Noise | Rápida | Baseline |
| TD3 | Off-Policy | Continua | Alta | Noise | Media | Benchmark |
| SAC | Off-Policy | Continua | Alta | Entropy | Media | SOTA Actual |

**Diagrama de Evolución**:
```
Policy Gradient (REINFORCE)
    ↓
A3C (Asynchronous Advantage Actor-Critic)
    ↓
TRPO (Trust Region Policy Optimization)
    ↓
┌─→ PPO (on-policy, versátil)
│
└─→ DQN
    ├─→ DDPG
    │   └─→ TD3 (twin critics, delayed updates)
    │
    └─→ SAC (maximum entropy)
```

In [ ]:
# Imports generales
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import gymnasium as gym
import matplotlib.pyplot as plt
from collections import deque
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)
torch.manual_seed(42)

print("Ambiente preparado:")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")
print(f"Gymnasium version: {gym.__version__}")

## 2. PPO: Proximal Policy Optimization Detallado (12 celdas)

### 2.1 Fundamentos de PPO

PPO mejora sobre TRPO eliminando el constraint de KL divergence complejo y usando un objetivo clipped más simple:

**Objective de PPO-Clip**:
```
L^CLIP(θ) = E[min(r_t(θ)A_t, clip(r_t(θ), 1-ε, 1+ε)A_t)]
```

donde:
- `r_t(θ) = π_θ(a|s) / π_θ_old(a|s)` es el ratio de probabilidades
- `A_t` es la ventaja (advantage)
- `ε` es el epsilon de clipping (típicamente 0.2)

**Clave**: El clipping previene que la política se actualice demasiado rápido, asegurando estabilidad sin requerir KL divergence.

**Características**:
1. GAE (Generalized Advantage Estimation) para estimar ventajas
2. Value function clipping opcional
3. Entropy bonus para exploración
4. Mini-batch training con múltiples epochs

In [ ]:
# Importar PPO desde el módulo local
import sys
sys.path.insert(0, '/home/user/Reinforcement-learning-guide')

from notebookx03_deep_rl.advanced.ppo import PPOAgent, evaluate_agent as ppo_evaluate, plot_training_results as ppo_plot

print("PPO Agent importado exitosamente")
print(f"Métodos disponibles: {[m for m in dir(PPOAgent) if not m.startswith('_')]}")

### 2.2 Entrenamiento PPO en LunarLander

LunarLander es un entorno clásico para control continuo:
- **Estado**: 8 dimensiones (posición, velocidad, ángulo, etc.)
- **Acciones**: 4 acciones discretas (sin motor, motor izq, motor central, motor der)
- **Objetivo**: Aterrizar con suavidad
- **Recompensa**: Hasta 200 puntos

In [ ]:
# Crear entorno LunarLander
env_lunar = gym.make('LunarLander-v2')
state_dim = env_lunar.observation_space.shape[0]
action_dim = env_lunar.action_space.n

print(f"Entorno: LunarLander-v2")
print(f"Dimensión de estado: {state_dim}")
print(f"Dimensión de acciones: {action_dim} (discretas)")
print(f"Espacio de acciones: {env_lunar.action_space}")

# Crear agente PPO
ppo_agent = PPOAgent(
    state_dim=state_dim,
    action_dim=action_dim,
    continuous=False,
    lr=3e-4,
    gamma=0.99,
    gae_lambda=0.95,
    epsilon_clip=0.2,
    value_clip=0.2,
    entropy_coef=0.01,
    n_epochs=4,
    batch_size=64,
    hidden_dims=[64, 64]
)

print(f"\nAgente PPO creado con {sum(p.numel() for p in ppo_agent.actor.parameters())} parámetros (actor)")

In [ ]:
# Entrenar PPO (versión reducida para demostración)
print("Iniciando entrenamiento PPO...")
history_ppo = ppo_agent.train(
    env=env_lunar,
    n_episodes=100,  # Reducido para tiempo de demostración
    max_steps=1000,
    update_interval=2048,
    print_every=20,
    save_every=0  # No guardar para ahorrar espacio
)

print("\nEntrenamiento completado!")

In [ ]:
# Visualizar resultados de PPO
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Recompensas
ax = axes[0, 0]
rewards = history_ppo['episode_rewards']
ax.plot(rewards, alpha=0.3, label='Episode Reward')
if len(rewards) > 10:
    window = min(20, len(rewards) // 5)
    moving_avg = np.convolve(rewards, np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(rewards)), moving_avg, label=f'MA({window})', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Reward')
ax.set_title('Recompensas PPO en LunarLander')
ax.legend()
ax.grid(True, alpha=0.3)

# Losses
ax = axes[0, 1]
ax.plot(history_ppo['actor_losses'], label='Actor Loss', alpha=0.6)
ax.plot(history_ppo['critic_losses'], label='Critic Loss', alpha=0.6)
ax.set_xlabel('Update')
ax.set_ylabel('Loss')
ax.set_title('Actor & Critic Losses')
ax.legend()
ax.grid(True, alpha=0.3)

# Clipping Fraction
ax = axes[1, 0]
ax.plot(history_ppo['clip_fractions'], alpha=0.6, label='Clip Fraction')
ax.axhline(y=0.2, color='r', linestyle='--', alpha=0.5, label='Target (20%)')
ax.set_xlabel('Update')
ax.set_ylabel('Fraction')
ax.set_title('Clipping Fraction (debería estar cerca de 20%)')
ax.legend()
ax.grid(True, alpha=0.3)

# Entropía
ax = axes[1, 1]
ax.plot(history_ppo['entropies'], alpha=0.6, label='Policy Entropy')
ax.set_xlabel('Update')
ax.set_ylabel('Entropy')
ax.set_title('Policy Entropy (exploración)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/tmp/ppo_results.png', dpi=100)
plt.show()

print(f"Última recompensa: {rewards[-1]:.2f}")
print(f"Recompensa promedio (últimas 20): {np.mean(rewards[-20:]):.2f}")

### 2.3 Objetivo Clipped de PPO - Análisis Detallado

El clipping es lo que diferencia PPO de TRPO. Veamos cómo funciona:

In [ ]:
# Visualizar el objetivo clipped de PPO
epsilon = 0.2
ratio = np.linspace(0.5, 1.5, 100)  # Probability ratio
advantage = 1.0  # Ventaja positiva

# Términos del objetivo
surr1 = ratio * advantage  # Sin clipping
clipped_ratio = np.clip(ratio, 1 - epsilon, 1 + epsilon)
surr2 = clipped_ratio * advantage  # Con clipping
ppo_loss = -np.minimum(surr1, surr2)  # Min de ambos

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Grafo 1: Comparación de términos
ax = axes[0]
ax.plot(ratio, surr1, label='r(θ) * A (sin clipping)', linewidth=2)
ax.plot(ratio, surr2, label='clip(r(θ), 1±ε) * A (con clipping)', linewidth=2)
ax.plot(ratio, ppo_loss, label='min(surr1, surr2) [PPO Loss]', linewidth=2.5, linestyle='--')
ax.axvline(x=1.0, color='red', linestyle=':', alpha=0.5, label='r = 1 (sin cambio)')
ax.axvline(x=1-epsilon, color='green', linestyle=':', alpha=0.5)
ax.axvline(x=1+epsilon, color='green', linestyle=':', alpha=0.5)
ax.set_xlabel('Probability Ratio r(θ)')
ax.set_ylabel('Loss')
ax.set_title('Objetivo Clipped de PPO (Advantage > 0)')
ax.legend()
ax.grid(True, alpha=0.3)

# Grafo 2: Importancia del epsilon
ax = axes[1]
epsilons = [0.1, 0.2, 0.3]
for eps in epsilons:
    clipped_ratio_eps = np.clip(ratio, 1 - eps, 1 + eps)
    surr2_eps = clipped_ratio_eps * advantage
    ppo_loss_eps = -np.minimum(surr1, surr2_eps)
    ax.plot(ratio, ppo_loss_eps, label=f'ε = {eps}', linewidth=2)
ax.axvline(x=1.0, color='red', linestyle=':', alpha=0.5)
ax.set_xlabel('Probability Ratio r(θ)')
ax.set_ylabel('Loss')
ax.set_title('Efecto de Epsilon en el Clipping')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Interpretación del Clipping:")
print(f"1. Cuando r(θ) < 1-ε: Política se hace menos probable (bad action) → gradiente cero")
print(f"2. Cuando 1-ε < r(θ) < 1: Aumentar probabilidad, pero limitado")
print(f"3. Cuando 1 < r(θ) < 1+ε: Disminuir probabilidad, pero limitado")
print(f"4. Cuando r(θ) > 1+ε: Política se hace demasiado probable → gradiente cero")

In [ ]:
# Evaluar política entrenada
print("Evaluando política PPO entrenada...")
mean_reward, std_reward = ppo_evaluate(ppo_agent, env_lunar, n_episodes=20, render=False)
print(f"\nRecompensa promedio: {mean_reward:.2f} ± {std_reward:.2f}")
print(f"Umbral de éxito en LunarLander: 200 puntos")
print(f"Performance: {'Excelente' if mean_reward > 200 else 'Buena' if mean_reward > 0 else 'Pobre'}")

env_lunar.close()

## 3. DDPG: Control Determinista Profundo (10 celdas)

### 3.1 Fundamentos de DDPG

DDPG (Deep Deterministic Policy Gradient) fue el primer algoritmo actor-critic para espacios continuos.

**Características Clave**:
1. **Actor Determinista**: μ(s) - mapea directamente estado a acción
2. **Critic Q-learning**: Q(s,a) - estima valor de par estado-acción
3. **Experience Replay**: rompe correlación temporal
4. **Target Networks**: copias lentas para estabilidad
5. **Exploration Noise**: Ornstein-Uhlenbeck o Gaussian

**Actualizaciones**:
```
Actual loss: Q(s,a) - (r + γQ'(s', μ'(s'))²
Actor loss: -E[Q(s, μ(s))]
```

In [ ]:
# Importar DDPG
from notebookx03_deep_rl.advanced.ddpg import DDPGAgent, evaluate_agent as ddpg_evaluate, plot_training_results as ddpg_plot

# Crear entorno Pendulum
env_pendulum = gym.make('Pendulum-v1')
state_dim = env_pendulum.observation_space.shape[0]
action_dim = env_pendulum.action_space.shape[0]

print(f"Entorno: Pendulum-v1")
print(f"Dimensión de estado: {state_dim}")
print(f"Dimensión de acciones: {action_dim} (continua)")
print(f"Rango de acciones: [{env_pendulum.action_space.low[0]:.2f}, {env_pendulum.action_space.high[0]:.2f}]")

# Crear agente DDPG
ddpg_agent = DDPGAgent(
    state_dim=state_dim,
    action_dim=action_dim,
    actor_lr=1e-4,
    critic_lr=1e-3,
    gamma=0.99,
    tau=0.001,
    buffer_size=100000,
    batch_size=64,
    noise_type='ou',
    noise_std=0.2,
    hidden_dims=[400, 300]
)

print(f"\nAgente DDPG creado")

In [ ]:
# Entrenar DDPG
print("Iniciando entrenamiento DDPG en Pendulum...")
history_ddpg = ddpg_agent.train(
    env=env_pendulum,
    n_episodes=50,  # Reducido para demostración
    max_steps=200,
    warmup_steps=1000,
    noise_decay=0.9995,
    min_noise=0.1,
    print_every=10,
    save_every=0
)

print("\nEntrenamiento completado!")

In [ ]:
# Visualizar DDPG
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Recompensas
ax = axes[0, 0]
rewards = history_ddpg['episode_rewards']
ax.plot(rewards, alpha=0.3, label='Episode Reward')
if len(rewards) > 5:
    window = min(10, len(rewards) // 5)
    moving_avg = np.convolve(rewards, np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(rewards)), moving_avg, label=f'MA({window})', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Reward')
ax.set_title('Recompensas DDPG en Pendulum')
ax.legend()
ax.grid(True, alpha=0.3)

# Losses
ax = axes[0, 1]
if history_ddpg['actor_losses']:
    ax.plot(history_ddpg['actor_losses'], label='Actor Loss', alpha=0.6)
    ax.plot(history_ddpg['critic_losses'], label='Critic Loss', alpha=0.6)
ax.set_xlabel('Episode')
ax.set_ylabel('Loss')
ax.set_title('Actor & Critic Losses')
ax.legend()
ax.grid(True, alpha=0.3)

# Q-values
ax = axes[1, 0]
if history_ddpg['q_values']:
    ax.plot(history_ddpg['q_values'], alpha=0.6, label='Mean Q-value')
ax.set_xlabel('Episode')
ax.set_ylabel('Q-value')
ax.set_title('Estimación de Valor')
ax.legend()
ax.grid(True, alpha=0.3)

# Lengths
ax = axes[1, 1]
lengths = history_ddpg['episode_lengths']
ax.plot(lengths, alpha=0.3, label='Episode Length')
if len(lengths) > 5:
    window = min(10, len(lengths) // 5)
    moving_avg = np.convolve(lengths, np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(lengths)), moving_avg, label=f'MA({window})', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Length')
ax.set_title('Duración de Episodios')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Última recompensa: {rewards[-1]:.2f}")
print(f"Mejor recompensa: {max(rewards):.2f}")

### 3.2 Exploración en DDPG: Ornstein-Uhlenbeck vs Gaussian

DDPG requiere ruido para exploración. Comparemos dos tipos:

In [ ]:
# Visualizar tipos de ruido
from notebookx03_deep_rl.advanced.ddpg import OrnsteinUhlenbeckNoise

# Generar ruido OU
ou_noise = OrnsteinUhlenbeckNoise(size=1, mu=0.0, theta=0.15, sigma=0.2)
ou_samples = [ou_noise.sample()[0] for _ in range(200)]

# Generar ruido Gaussiano
gaussian_samples = np.random.randn(200) * 0.2

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Ruido OU
ax = axes[0]
ax.plot(ou_samples, alpha=0.7, label='OU Noise')
ax.set_title('Ornstein-Uhlenbeck Noise (correlacionado temporalmente)')
ax.set_xlabel('Timestep')
ax.set_ylabel('Noise Value')
ax.grid(True, alpha=0.3)
ax.legend()

# Ruido Gaussiano
ax = axes[1]
ax.plot(gaussian_samples, alpha=0.7, label='Gaussian Noise', color='orange')
ax.set_title('Gaussian Noise (independiente)')
ax.set_xlabel('Timestep')
ax.set_ylabel('Noise Value')
ax.grid(True, alpha=0.3)
ax.legend()

plt.tight_layout()
plt.show()

print("Análisis de Ruido:")
print(f"\nOU Noise:")
print(f"  - Correlación temporal (smooth)")
print(f"  - Útil para problemas con inercia física")
print(f"  - Media: {np.mean(ou_samples):.4f}, Std: {np.std(ou_samples):.4f}")
print(f"\nGaussian Noise:")
print(f"  - Independiente, aleatorio puro")
print(f"  - Más simple, igual de efectivo en muchos casos")
print(f"  - Media: {np.mean(gaussian_samples):.4f}, Std: {np.std(gaussian_samples):.4f}")

In [ ]:
# Evaluar DDPG
print("Evaluando política DDPG...")
mean_reward, std_reward = ddpg_evaluate(ddpg_agent, env_pendulum, n_episodes=10)
print(f"Recompensa promedio: {mean_reward:.2f} ± {std_reward:.2f}")
print(f"Objetivo en Pendulum: > -300 puntos")

env_pendulum.close()

## 4. TD3: Twin Delayed DDPG (10 celdas)

### 4.1 Tres Mejoras Clave de TD3

TD3 resuelve problemas de inestabilidad en DDPG:

**1. Twin Q-networks (Clipped Double Q-learning)**
```
Target = min(Q₁'(s', a'), Q₂'(s', a'))
```
- Reduce sobreestimación que causa inestabilidad

**2. Delayed Policy Updates**
- Actor se actualiza cada d steps
- Critics se actualizan en cada step
- Reduce varianza del gradiente de política

**3. Target Policy Smoothing**
```
a' = clip(μ'(s') + ε, action_min, action_max)
ε ~ N(0, σ²)
```
- Suaviza superficie de Q-value
- Más robusto a errores de aproximación

In [ ]:
# Importar TD3
from notebookx03_deep_rl.advanced.td3 import TD3Agent, evaluate_agent as td3_evaluate

# Crear entorno
env_td3 = gym.make('Pendulum-v1')
state_dim = env_td3.observation_space.shape[0]
action_dim = env_td3.action_space.shape[0]
max_action = float(env_td3.action_space.high[0])

print(f"Entorno: Pendulum-v1 (para TD3)")

# Crear agente TD3
td3_agent = TD3Agent(
    state_dim=state_dim,
    action_dim=action_dim,
    max_action=max_action,
    actor_lr=3e-4,
    critic_lr=3e-4,
    gamma=0.99,
    tau=0.005,
    policy_noise=0.2,
    noise_clip=0.5,
    policy_delay=2,
    buffer_size=1000000,
    batch_size=256,
    exploration_noise=0.1,
    hidden_dims=[256, 256]
)

print(f"\nAgente TD3 creado con Twin Critics")

In [ ]:
# Entrenar TD3
print("Iniciando entrenamiento TD3...")
history_td3 = td3_agent.train(
    env=env_td3,
    n_episodes=50,
    max_steps=200,
    warmup_steps=1000,
    noise_decay=0.999,
    min_noise=0.1,
    print_every=10,
    save_every=0
)

print("Entrenamiento completado!")

In [ ]:
# Visualizar TD3
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Recompensas
ax = axes[0, 0]
rewards = history_td3['episode_rewards']
ax.plot(rewards, alpha=0.3, label='Episode Reward')
if len(rewards) > 5:
    window = min(10, len(rewards) // 5)
    moving_avg = np.convolve(rewards, np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(rewards)), moving_avg, label=f'MA({window})', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Reward')
ax.set_title('Recompensas TD3')
ax.legend()
ax.grid(True, alpha=0.3)

# Losses
ax = axes[0, 1]
if history_td3['actor_losses'] and history_td3['critic_losses']:
    ax.plot(history_td3['actor_losses'], label='Actor Loss', alpha=0.6)
    ax.plot(history_td3['critic_losses'], label='Critic Loss', alpha=0.6)
ax.set_xlabel('Episode')
ax.set_ylabel('Loss')
ax.set_title('Losses')
ax.legend()
ax.grid(True, alpha=0.3)

# Twin Q-values
ax = axes[0, 2]
if history_td3['q1_values'] and history_td3['q2_values']:
    ax.plot(history_td3['q1_values'], label='Q1', alpha=0.6)
    ax.plot(history_td3['q2_values'], label='Q2', alpha=0.6)
ax.set_xlabel('Episode')
ax.set_ylabel('Q-value')
ax.set_title('Twin Q-values (reducen sobreestimación)')
ax.legend()
ax.grid(True, alpha=0.3)

# Q1 - Q2 (debe ser cercano a 0)
ax = axes[1, 0]
if history_td3['q1_values'] and history_td3['q2_values']:
    q_diff = np.array(history_td3['q1_values']) - np.array(history_td3['q2_values'])
    ax.plot(q_diff, alpha=0.6, label='Q1 - Q2')
    ax.axhline(y=0, color='r', linestyle='--', alpha=0.3)
ax.set_xlabel('Episode')
ax.set_ylabel('Q Difference')
ax.set_title('Diferencia Q1-Q2 (convergencia)')
ax.legend()
ax.grid(True, alpha=0.3)

# Duración
ax = axes[1, 1]
lengths = history_td3['episode_lengths']
ax.plot(lengths, alpha=0.3, label='Episode Length')
if len(lengths) > 5:
    window = min(10, len(lengths) // 5)
    moving_avg = np.convolve(lengths, np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(lengths)), moving_avg, label=f'MA({window})', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Length')
ax.set_title('Duración de Episodios')
ax.legend()
ax.grid(True, alpha=0.3)

# Distribuición de recompensas
ax = axes[1, 2]
if len(rewards) >= 10:
    recent = rewards[-min(30, len(rewards)):]
    ax.hist(recent, bins=15, alpha=0.7, edgecolor='black')
    ax.axvline(np.mean(recent), color='r', linestyle='--', linewidth=2, label=f'Mean: {np.mean(recent):.1f}')
ax.set_xlabel('Reward')
ax.set_ylabel('Frequency')
ax.set_title('Distribución de Recompensas')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Última recompensa: {rewards[-1]:.2f}")
print(f"Mejor recompensa: {max(rewards):.2f}")
print(f"Promedio (últimas 10): {np.mean(rewards[-10:]):.2f}")

### 4.2 Mejoras de TD3 vs DDPG

Visualicemos el impacto de las mejoras:

In [ ]:
# Comparar DDPG vs TD3 (mismo número de episodios)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Recompensas comparativas
ax = axes[0]
ax.plot(history_ddpg['episode_rewards'], label='DDPG', alpha=0.7, linewidth=1.5)
ax.plot(history_td3['episode_rewards'], label='TD3', alpha=0.7, linewidth=1.5)
ax.set_xlabel('Episode')
ax.set_ylabel('Reward')
ax.set_title('Comparación de Recompensas: DDPG vs TD3')
ax.legend()
ax.grid(True, alpha=0.3)

# Estadísticas
ax = axes[1]
methods = ['DDPG', 'TD3']
means = [np.mean(history_ddpg['episode_rewards']), np.mean(history_td3['episode_rewards'])]
stds = [np.std(history_ddpg['episode_rewards']), np.std(history_td3['episode_rewards'])]
x = np.arange(len(methods))
bars = ax.bar(x, means, yerr=stds, capsize=10, alpha=0.7, edgecolor='black')
ax.set_ylabel('Recompensa Promedio')
ax.set_title('Recompensas Promedio y Desviación Estándar')
ax.set_xticks(x)
ax.set_xticklabels(methods)
ax.grid(True, alpha=0.3, axis='y')

# Añadir valores en las barras
for i, (mean, std) in enumerate(zip(means, stds)):
    ax.text(i, mean + std + 5, f'{mean:.1f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"DDPG - Media: {np.mean(history_ddpg['episode_rewards']):.2f}, Std: {np.std(history_ddpg['episode_rewards']):.2f}")
print(f"TD3  - Media: {np.mean(history_td3['episode_rewards']):.2f}, Std: {np.std(history_td3['episode_rewards']):.2f}")

In [ ]:
# Evaluar TD3
print("Evaluando política TD3...")
mean_reward, std_reward = td3_evaluate(td3_agent, env_td3, n_episodes=10)
print(f"Recompensa promedio: {mean_reward:.2f} ± {std_reward:.2f}")

env_td3.close()

## 5. SAC: Soft Actor-Critic (12 celdas)

### 5.1 Maximum Entropy Reinforcement Learning

SAC introduce el concepto de máxima entropía, maximizando tanto recompensas como incertidumbre:

**Objetivo**:
```
J(π) = E[Σ γᵗ(r_t + αH(π(·|s_t)))]
```

donde:
- r_t: recompensa
- α: temperature (controlada automáticamente)
- H(π): entropía de la política

**Ventajas de máxima entropía**:
1. Exploración natural
2. Políticas multimodales (múltiples soluciones)
3. Robustez a cambios en el entorno
4. Mejor distanciamiento de política anterior (para fine-tuning)

In [ ]:
# Importar SAC
from notebookx03_deep_rl.advanced.sac import SACAgent, evaluate_agent as sac_evaluate

# Crear entorno
env_sac = gym.make('Pendulum-v1')
state_dim = env_sac.observation_space.shape[0]
action_dim = env_sac.action_space.shape[0]

print(f"Entorno: Pendulum-v1 (para SAC)")

# Crear agente SAC
sac_agent = SACAgent(
    state_dim=state_dim,
    action_dim=action_dim,
    actor_lr=3e-4,
    critic_lr=3e-4,
    alpha_lr=3e-4,
    gamma=0.99,
    tau=0.005,
    alpha=0.2,
    auto_tune=True,
    target_entropy=None,  # Será -action_dim
    buffer_size=1000000,
    batch_size=256,
    hidden_dims=[256, 256]
)

print(f"\nAgente SAC creado con auto-tuning de temperatura")

In [ ]:
# Entrenar SAC
print("Iniciando entrenamiento SAC con auto-tuning de α...")
history_sac = sac_agent.train(
    env=env_sac,
    n_episodes=50,
    max_steps=200,
    warmup_steps=1000,
    updates_per_step=1,
    print_every=10,
    save_every=0
)

print("\nEntrenamiento completado!")

In [ ]:
# Visualizar SAC
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Recompensas
ax = axes[0, 0]
rewards = history_sac['episode_rewards']
ax.plot(rewards, alpha=0.3, label='Episode Reward')
if len(rewards) > 5:
    window = min(10, len(rewards) // 5)
    moving_avg = np.convolve(rewards, np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(rewards)), moving_avg, label=f'MA({window})', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Reward')
ax.set_title('Recompensas SAC')
ax.legend()
ax.grid(True, alpha=0.3)

# Losses
ax = axes[0, 1]
if history_sac['actor_losses'] and history_sac['critic_losses']:
    ax.plot(history_sac['actor_losses'], label='Actor Loss', alpha=0.6)
    ax.plot(history_sac['critic_losses'], label='Critic Loss', alpha=0.6)
ax.set_xlabel('Episode')
ax.set_ylabel('Loss')
ax.set_title('Actor & Critic Losses')
ax.legend()
ax.grid(True, alpha=0.3)

# Temperatura α
ax = axes[0, 2]
if history_sac['alphas']:
    ax.plot(history_sac['alphas'], alpha=0.6, label='α (auto-tuned)')
    ax.axhline(y=0.2, color='r', linestyle='--', alpha=0.3, label='Initial α')
ax.set_xlabel('Episode')
ax.set_ylabel('α')
ax.set_title('Auto-tuning de Temperatura')
ax.legend()
ax.grid(True, alpha=0.3)

# Entropía
ax = axes[1, 0]
if history_sac['entropies']:
    ax.plot(history_sac['entropies'], alpha=0.6, label='Policy Entropy')
    target_entropy = -action_dim  # Objetivo de entropía
    ax.axhline(y=target_entropy, color='r', linestyle='--', alpha=0.3, label=f'Target ({target_entropy:.2f})')
ax.set_xlabel('Episode')
ax.set_ylabel('Entropy')
ax.set_title('Policy Entropy (exploración)')
ax.legend()
ax.grid(True, alpha=0.3)

# Twin Q-values
ax = axes[1, 1]
if history_sac['q1_values'] and history_sac['q2_values']:
    ax.plot(history_sac['q1_values'], label='Q1', alpha=0.6)
    ax.plot(history_sac['q2_values'], label='Q2', alpha=0.6)
ax.set_xlabel('Episode')
ax.set_ylabel('Q-value')
ax.set_title('Twin Q-values')
ax.legend()
ax.grid(True, alpha=0.3)

# Duración
ax = axes[1, 2]
lengths = history_sac['episode_lengths']
ax.plot(lengths, alpha=0.3, label='Episode Length')
if len(lengths) > 5:
    window = min(10, len(lengths) // 5)
    moving_avg = np.convolve(lengths, np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(lengths)), moving_avg, label=f'MA({window})', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Length')
ax.set_title('Duración de Episodios')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Última recompensa: {rewards[-1]:.2f}")
print(f"Mejor recompensa: {max(rewards):.2f}")
print(f"Promedio: {np.mean(rewards):.2f}")

### 5.2 Auto-tuning de Temperatura en SAC

SAC ajusta automáticamente α para mantener entropía objetivo. Veamos cómo funciona:

In [ ]:
# Visualizar relación entre α, entropía y recompensa
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Relación entre α y Entropía
ax = axes[0]
ax.plot(history_sac['alphas'], label='Temperature α', alpha=0.6, linewidth=2)
ax2 = ax.twinx()
ax2.plot(history_sac['entropies'], label='Policy Entropy', color='orange', alpha=0.6, linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('α', color='blue')
ax2.set_ylabel('Entropy', color='orange')
ax.set_title('Auto-tuning: α controla Entropía')
ax.grid(True, alpha=0.3)
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

# Scatter plot: α vs Entropía
ax = axes[1]
if history_sac['alphas'] and history_sac['entropies']:
    scatter = ax.scatter(history_sac['alphas'], history_sac['entropies'], 
                         c=range(len(history_sac['alphas'])), cmap='viridis', alpha=0.6, s=50)
    target_entropy = -action_dim
    ax.axhline(y=target_entropy, color='r', linestyle='--', alpha=0.5, linewidth=2, label=f'Target entropía ({target_entropy:.2f})')
    ax.set_xlabel('Temperature α')
    ax.set_ylabel('Policy Entropy')
    ax.set_title('Relación entre α y Entropía')
    ax.legend()
    ax.grid(True, alpha=0.3)
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('Episode')

plt.tight_layout()
plt.show()

print("Interpretación del Auto-tuning:")
print(f"\n1. Si entropía < objetivo: α aumenta (fomentar exploración)")
print(f"2. Si entropía > objetivo: α disminuye (enfocarse en recompensas)")
print(f"3. Temperatura final: {history_sac['alphas'][-1]:.4f}")
print(f"4. Entropía final: {history_sac['entropies'][-1]:.4f}")
print(f"5. Entropía objetivo: {-action_dim:.4f}")

In [ ]:
# Evaluar SAC
print("Evaluando política SAC...")
mean_reward, std_reward = sac_evaluate(sac_agent, env_sac, n_episodes=10)
print(f"Recompensa promedio: {mean_reward:.2f} ± {std_reward:.2f}")

env_sac.close()

## 6. Comparación Completa de los 4 Algoritmos (8 celdas)

### 6.1 Análisis de Rendimiento Comparativo

In [ ]:
# Preparar datos para comparación
algorithms = {
    'PPO': {
        'rewards': history_ppo['episode_rewards'],
        'color': 'blue',
        'type': 'On-policy'
    },
    'DDPG': {
        'rewards': history_ddpg['episode_rewards'],
        'color': 'orange',
        'type': 'Off-policy (deterministic)'
    },
    'TD3': {
        'rewards': history_td3['episode_rewards'],
        'color': 'green',
        'type': 'Off-policy (deterministic)'
    },
    'SAC': {
        'rewards': history_sac['episode_rewards'],
        'color': 'red',
        'type': 'Off-policy (stochastic)'
    }
}

# Comparación de recompensas
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Trayectorias de entrenamiento
ax = axes[0, 0]
for algo, data in algorithms.items():
    rewards = data['rewards']
    ax.plot(rewards, label=algo, color=data['color'], alpha=0.6)
ax.set_xlabel('Episode')
ax.set_ylabel('Reward')
ax.set_title('Trayectorias de Entrenamiento')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Media móvil (últimos 10 episodios)
ax = axes[0, 1]
for algo, data in algorithms.items():
    rewards = data['rewards']
    window = min(10, len(rewards) // 3)
    if window > 1:
        moving_avg = np.convolve(rewards, np.ones(window)/window, mode='valid')
        ax.plot(range(window-1, len(rewards)), moving_avg, label=algo, 
               color=data['color'], alpha=0.7, linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Reward (Moving Average)')
ax.set_title('Media Móvil (Convergencia)')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Estadísticas
ax = axes[1, 0]
stats_data = {}
for algo, data in algorithms.items():
    rewards = data['rewards']
    stats_data[algo] = {
        'mean': np.mean(rewards),
        'std': np.std(rewards),
        'max': np.max(rewards),
        'min': np.min(rewards)
    }

x = np.arange(len(algorithms))
means = [stats_data[algo]['mean'] for algo in algorithms.keys()]
stds = [stats_data[algo]['std'] for algo in algorithms.keys()]
colors = [data['color'] for data in algorithms.values()]

bars = ax.bar(x, means, yerr=stds, capsize=10, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
ax.set_ylabel('Recompensa Promedio')
ax.set_title('Rendimiento Promedio ± Desviación Estándar')
ax.set_xticks(x)
ax.set_xticklabels(algorithms.keys())
ax.grid(True, alpha=0.3, axis='y')

# Añadir valores
for i, (mean, std) in enumerate(zip(means, stds)):
    ax.text(i, mean + std + 2, f'{mean:.1f}', ha='center', va='bottom', fontweight='bold')

# 4. Box plot
ax = axes[1, 1]
box_data = [data['rewards'] for data in algorithms.values()]
bp = ax.boxplot(box_data, labels=list(algorithms.keys()), patch_artist=True)
for patch, data in zip(bp['boxes'], algorithms.values()):
    patch.set_facecolor(data['color'])
    patch.set_alpha(0.7)
ax.set_ylabel('Reward')
ax.set_title('Distribución de Recompensas')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("Estadísticas de Rendimiento:")
print("\n{:<10} {:<12} {:<12} {:<12} {:<12}".format("Algoritmo", "Media", "Std", "Max", "Min"))
print("-" * 58)
for algo, stats in stats_data.items():
    print("{:<10} {:<12.2f} {:<12.2f} {:<12.2f} {:<12.2f}".format(
        algo, stats['mean'], stats['std'], stats['max'], stats['min']))

### 6.2 Tabla Comparativa Detallada

In [ ]:
# Crear tabla comparativa
import pandas as pd

comparison_data = {
    'Algoritmo': ['PPO', 'DDPG', 'TD3', 'SAC'],
    'Tipo': ['On-Policy', 'Off-Policy', 'Off-Policy', 'Off-Policy'],
    'Acción': ['Discreta/Continua', 'Continua', 'Continua', 'Continua'],
    'Política': ['Estocástica', 'Determinista', 'Determinista', 'Estocástica'],
    'Estabilidad': ['Media', 'Baja', 'Alta', 'Alta'],
    'Exploración': ['Entropy', 'Noise', 'Noise', 'Max Entropy'],
    'Sample Eff.': ['Baja', 'Alta', 'Alta', 'Alta'],
    'Complejidad': ['Baja', 'Media', 'Media-Alta', 'Alta'],
    'Mejor Para': ['Robotics, Vision', 'Baseline', 'Benchmark', 'SOTA Apps'],
    'Año': [2017, 2016, 2018, 2018]
}

df_comparison = pd.DataFrame(comparison_data)
print("\nTABLA COMPARATIVA - ALGORITMOS SOTA")
print("="*100)
print(df_comparison.to_string(index=False))
print("="*100)

### 6.3 Selección de Algoritmo Según Problema

In [ ]:
# Matriz de decisión
print("\nGUÍA DE SELECCIÓN DE ALGORITMO")
print("="*80)

selection_guide = {
    'PPO': {
        'description': 'Proximal Policy Optimization',
        'pros': [
            '✓ Versátil (discreto y continuo)',
            '✓ Fácil de implementar y tunear',
            '✓ Estable sin target networks',
            '✓ SOTA en visión y robotics',
            '✓ Usado en ChatGPT (RLHF)'
        ],
        'cons': [
            '✗ On-policy: muestra ineficiente',
            '✗ Requiere muchos samples',
            '✗ No ideal para off-line RL'
        ],
        'use_case': 'Cuando tienes abundancia de data y quieres entrenamiento estable'
    },
    'DDPG': {
        'description': 'Deep Deterministic Policy Gradient',
        'pros': [
            '✓ Primer algoritmo para control continuo',
            '✓ Sample efficient (off-policy)',
            '✓ Rápido de entrenar',
            '✓ Buena baseline'
        ],
        'cons': [
            '✗ Inestable en problemas complejos',
            '✗ Sensible a hiperparámetros',
            '✗ Puede diverger'
        ],
        'use_case': 'Experimentos iniciales, problemas simples continuos'
    },
    'TD3': {
        'description': 'Twin Delayed DDPG',
        'pros': [
            '✓ Más estable que DDPG',
            '✓ Mejoras demostradas empíricamente',
            '✓ Buen benchmark',
            '✓ Twin critics reducen sesgo'
        ],
        'cons': [
            '✗ Más complejo que DDPG',
            '✗ Más lento que DDPG',
            '✗ Todavía puede ser inestable'
        ],
        'use_case': 'Comparación de baselines, cuando necesitas mejor rendimiento que DDPG'
    },
    'SAC': {
        'description': 'Soft Actor-Critic',
        'pros': [
            '✓ SOTA actual para control continuo',
            '✓ Auto-tuning de temperatura',
            '✓ Robusto y estable',
            '✓ Excelente para aplicaciones reales',
            '✓ Políticas exploratorias'
        ],
        'cons': [
            '✗ Más complejo de implementar',
            '✗ Más parámetros a tunear',
            '✗ Computacionalmente más costoso'
        ],
        'use_case': 'Aplicaciones en producción, cuando necesitas lo mejor disponible'
    }
}

for algo, info in selection_guide.items():
    print(f"\n{algo}: {info['description']}")
    print("-" * 80)
    print("\nVentajas:")
    for pro in info['pros']:
        print(f"  {pro}")
    print("\nDesventajas:")
    for con in info['cons']:
        print(f"  {con}")
    print(f"\nCuándo usar: {info['use_case']}")
    print()

### 6.4 Convergencia y Dinamics de Aprendizaje

In [ ]:
# Analizar velocidad de convergencia
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. Tasa de mejora
ax = axes[0, 0]
for algo, data in algorithms.items():
    rewards = data['rewards']
    # Calcular mejora acumulativa
    improvement = np.cumsum(np.diff(rewards, prepend=rewards[0]))
    ax.plot(improvement, label=algo, color=data['color'], alpha=0.7, linewidth=1.5)
ax.set_xlabel('Episode')
ax.set_ylabel('Cumulative Improvement')
ax.set_title('Tasa de Mejora Acumulativa')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Velocidad de convergencia a la media
ax = axes[0, 1]
for algo, data in algorithms.items():
    rewards = np.array(data['rewards'])
    final_mean = np.mean(rewards[-10:])
    # Distancia a la media final
    distance = np.abs(rewards - final_mean)
    # Suavizar
    window = max(1, len(distance) // 10)
    smoothed = np.convolve(distance, np.ones(window)/window, mode='same')
    ax.plot(smoothed, label=algo, color=data['color'], alpha=0.7, linewidth=1.5)
ax.set_xlabel('Episode')
ax.set_ylabel('|Reward - Final Mean|')
ax.set_title('Convergencia a Media Final')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Variabilidad (rolling std)
ax = axes[1, 0]
for algo, data in algorithms.items():
    rewards = data['rewards']
    window = max(1, len(rewards) // 10)
    rolling_std = pd.Series(rewards).rolling(window=window, center=True).std().values
    ax.plot(rolling_std, label=algo, color=data['color'], alpha=0.7, linewidth=1.5)
ax.set_xlabel('Episode')
ax.set_ylabel('Rolling Std Dev')
ax.set_title('Variabilidad (Rolling Std Dev)')
ax.legend()
ax.grid(True, alpha=0.3)

# 4. Cambio de episodio a episodio
ax = axes[1, 1]
for algo, data in algorithms.items():
    rewards = np.array(data['rewards'])
    changes = np.abs(np.diff(rewards))
    # Suavizar
    window = max(1, len(changes) // 10)
    smoothed = np.convolve(changes, np.ones(window)/window, mode='same')
    ax.plot(smoothed, label=algo, color=data['color'], alpha=0.7, linewidth=1.5)
ax.set_xlabel('Episode')
ax.set_ylabel('|Δ Reward|')
ax.set_title('Cambios Episodio-a-Episodio (Volatilidad)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Análisis de Convergencia:")
print("\nVelocidad de convergencia (menor es mejor):")
for algo, data in algorithms.items():
    rewards = np.array(data['rewards'])
    final_mean = np.mean(rewards[-10:])
    # Episodios hasta alcanzar 90% de la media final
    threshold = final_mean * 0.9
    reached_idx = np.where(rewards > threshold)[0]
    if len(reached_idx) > 0:
        episodes_to_converge = reached_idx[0]
    else:
        episodes_to_converge = len(rewards)
    print(f"{algo}: ~{episodes_to_converge} episodios para alcanzar 90% de media final")

## 7. Ejercicios Prácticos (5 celdas)

### 7.1 Ejercicio 1: Ajuste de Hiperparámetros

**Objetivo**: Experimentar con hiperparámetros de PPO para mejorar rendimiento en LunarLander

**Tarea**: 
1. Crea un agente PPO con diferentes valores de `epsilon_clip` (0.1, 0.2, 0.3)
2. Entrena cada uno por 50 episodios
3. Compara sus recompensas finales
4. ¿Cuál es el mejor valor? ¿Por qué?

**Hints**:
- epsilon_clip controla cuánto puede cambiar la política por update
- Valores más altos = cambios más grandes
- Trade-off entre velocidad y estabilidad

In [ ]:
# Ejercicio 1: Tu código aquí
print("Ejercicio 1: Ajuste de Hiperparámetros en PPO")
print("="*50)
print("\nImplementa una búsqueda de hiperparámetros para epsilon_clip")
print("\n# TODO: Modifica los valores de epsilon_clip y entrena")
print("# TODO: Compara los resultados")

# Solución plantilla:
epsilon_values = [0.1, 0.2, 0.3]
results = {}

for eps in epsilon_values:
    print(f"\nEntrenando con epsilon_clip={eps}...")
    # env = gym.make('LunarLander-v2')
    # agent = PPOAgent(..., epsilon_clip=eps, ...)
    # history = agent.train(...)
    # results[eps] = np.mean(history['episode_rewards'][-10:])
    # print(f"Recompensa media (últimas 10): {results[eps]:.2f}")

print("\n# Análisis: ¿Cuál fue el mejor epsilon_clip?")

### 7.2 Ejercicio 2: Comparación de Políticas (Determinista vs Estocástica)

**Objetivo**: Entender diferencias entre políticas deterministas y estocásticas

**Tarea**:
1. Toma un agente DDPG (determinista) entrenado
2. Ejecuta 10 episodios y registra las acciones
3. Compara con SAC (estocástica) ejecutando 10 episodios
4. ¿Cuáles son las diferencias en variabilidad de acciones?

**Análisis**:
- Determinista: misma acción para mismo estado
- Estocástica: variación incluso para mismo estado

In [ ]:
# Ejercicio 2: Comparar Determinista vs Estocástica
print("Ejercicio 2: Políticas Deterministas vs Estocásticas")
print("="*50)
print("\n# TODO: Extrae acciones de DDPG y SAC para el mismo estado")
print("# TODO: Visualiza la variabilidad")
print("# TODO: Analiza las diferencias")

print("\nObservaciones esperadas:")
print("- DDPG: acciones muy similares para repetir el mismo estado")
print("- SAC: acciones varían incluso para el mismo estado")
print("- SAC es más exploratoria naturalmente")

### 7.3 Ejercicio 3: Eficiencia de Muestras (Sample Efficiency)

**Objetivo**: Comparar eficiencia de muestras entre algoritmos

**Tarea**:
1. Para cada algoritmo, registra el número total de steps de ambiente ejecutados
2. Calcula: recompensa / número de steps
3. ¿Cuál es más eficiente?
4. ¿Por qué algunos son más eficientes que otros?

In [ ]:
# Ejercicio 3: Sample Efficiency
print("Ejercicio 3: Eficiencia de Muestras")
print("="*50)

total_steps = {
    'PPO': sum(history_ppo['episode_lengths']),
    'DDPG': sum(history_ddpg['episode_lengths']),
    'TD3': sum(history_td3['episode_lengths']),
    'SAC': sum(history_sac['episode_lengths'])
}

final_rewards = {
    'PPO': np.mean(history_ppo['episode_rewards'][-10:]),
    'DDPG': np.mean(history_ddpg['episode_rewards'][-10:]),
    'TD3': np.mean(history_td3['episode_rewards'][-10:]),
    'SAC': np.mean(history_sac['episode_rewards'][-10:])
}

print("\nEficiencia de Muestras (Reward / Steps):")
print("-" * 50)
for algo in ['PPO', 'DDPG', 'TD3', 'SAC']:
    efficiency = final_rewards[algo] / total_steps[algo] if total_steps[algo] > 0 else 0
    print(f"{algo:5} - Steps: {total_steps[algo]:5}, Reward: {final_rewards[algo]:7.2f}, Efficiency: {efficiency:.6f}")

print("\n# Análisis:")
print("# On-policy (PPO) vs Off-policy (DDPG, TD3, SAC)")
print("# Off-policy reutiliza experiencias del replay buffer")

### 7.4 Ejercicio 4: Transferencia de Política (Meta-Learning Prep)

**Objetivo**: Preparación para meta-learning - transferir política

**Tarea**:
1. Entrena un agente PPO en LunarLander
2. Guarda los pesos
3. Crea un nuevo agente con los mismos pesos
4. Fine-tunea en un entorno similar (CartPole)
5. Compara con entrenamiento desde cero

**Concepto**: Meta-learning = aprender a aprender

In [ ]:
# Ejercicio 4: Transfer Learning / Meta-Learning Prep
print("Ejercicio 4: Transferencia de Política (Prep para Meta-Learning)")
print("="*60)

print("\n# TODO: Guarda los pesos del agente entrenado")
print("# TODO: Crea un nuevo agente y carga los pesos")
print("# TODO: Fine-tunea en un entorno nuevo")
print("# TODO: Compara velocidad de aprendizaje")

print("\nCódigo de ejemplo:")
print("""
# Guardar
ppo_agent.save('ppo_lunarlander.pth')

# Cargar en nuevo agente
ppo_agent_new = PPOAgent(...)
ppo_agent_new.load('ppo_lunarlander.pth')

# Fine-tune en CartPole
history_transfer = ppo_agent_new.train(env_cartpole, n_episodes=50, ...)

# Comparar con entrenamiento desde cero
""")

### 7.5 Ejercicio 5: Diagnóstico de Problemas

**Objetivo**: Aprender a diagnosticar problemas comunes en RL

**Problemas comunes y soluciones**:

In [ ]:
# Ejercicio 5: Diagnóstico de Problemas
print("Ejercicio 5: Diagnóstico de Problemas Comunes en RL")
print("="*60)

diagnosis = {
    'Problema': [
        'Recompensas no mejoran',
        'Loss sube y sube',
        'Política diverge',
        'Entrenamiento muy lento',
        'Q-values muy negativos',
        'Mucho ruido en recompensas'
    ],
    'Síntoma': [
        'rewards = constante plana',
        'loss > 100 y creciendo',
        'gradientes NaN o infinitos',
        'poca mejora en 1000+ steps',
        'q_values < -1000',
        'alta varianza en rewards'
    ],
    'Posible Causa': [
        'Entorno demasiado complejo / learning rate bajo',
        'Reward scaling / gradient explosion',
        'Batch norm issues / learning rate alto',
        'Hiperparámetros conservadores',
        'Reward scaling / entorno mal normalizado',
        'Policy exploration insuficiente'
    ],
    'Solución': [
        'Aumentar learning rate / Simplificar entorno / Reward shaping',
        'Reducir learning rate / Normalizar rewards / Gradient clipping',
        'Reducir lr / Usar batch norm / Gradient clipping',
        'Aumentar batch size / lr / epsilon_clip / entropy coef',
        'Normalizar rewards / Usar smaller action magnitudes',
        'Aumentar entropy coef / Exploration noise / Target entropy'
    ]
}

import pandas as pd
df_diagnosis = pd.DataFrame(diagnosis)
print("\n" + df_diagnosis.to_string(index=False))

print("\n\nTu turno: ¿Qué diagnóstico harías para cada caso?")

## 8. Conclusiones y Próximos Pasos (4 celdas)

### 8.1 Resumen de Conceptos Clave

In [ ]:
print("RESUMEN DE CONCEPTOS CLAVE")
print("="*80)

summary = {
    "PPO": {
        "Insight Principal": "Clipping previene updates demasiado grandes",
        "Ecuación Clave": "L = E[min(rA, clip(r,1±ε)A)]",
        "Fortaleza": "Versatilidad y estabilidad",
        "Debilidad": "On-policy: muestra ineficiente",
        "Casos de Uso": "Robotics, Vision, RLHF"
    },
    "DDPG": {
        "Insight Principal": "Actor determinista + ruido = exploración eficiente",
        "Ecuación Clave": "Actor: -E[Q(s, μ(s))]",
        "Fortaleza": "Primer algoritmo para control continuo",
        "Debilidad": "Inestable, sensible a hiperparámetros",
        "Casos de Uso": "Baseline, problemas simples"
    },
    "TD3": {
        "Insight Principal": "Twin critics + delayed updates = robustez",
        "Ecuación Clave": "Target = min(Q₁', Q₂')",
        "Fortaleza": "Estable y robusto",
        "Debilidad": "Más complejo, más lento",
        "Casos de Uso": "Benchmarks, comparación de baselines"
    },
    "SAC": {
        "Insight Principal": "Máxima entropía = exploración + robustez automática",
        "Ecuación Clave": "J = E[r + αH(π)] con α auto-tuning",
        "Fortaleza": "SOTA, auto-tuning, robusto",
        "Debilidad": "Más parámetros, computacionalmente costoso",
        "Casos de Uso": "Aplicaciones en producción, SOTA"
    }
}

for algo, props in summary.items():
    print(f"\n{algo}")
    print("-" * 80)
    for key, value in props.items():
        print(f"  {key:<25}: {value}")

### 8.2 Tendencias Futuras y Meta-Learning

In [ ]:
print("FUTURO DEL APRENDIZAJE POR REFUERZO")
print("="*80)

future_topics = {
    "Meta-Learning": {
        "Descripción": "Aprender a aprender - quick adaptation a nuevas tareas",
        "Métodos": ["MAML (Model-Agnostic Meta-Learning)", "Few-shot RL", "Multi-task RL"],
        "Relevancia": "Crítica para sistemas generales"
    },
    "Offline RL": {
        "Descripción": "Aprender de datos recolectados sin interacción online",
        "Métodos": ["Conservative Q-learning", "IQL", "CQL"],
        "Relevancia": "Esencial para aplicaciones reales"
    },
    "Hierarchical RL": {
        "Descripción": "Aprendizaje con abstracciones de múltiples niveles",
        "Métodos": ["HRL", "Options Framework", "Feudal Networks"],
        "Relevancia": "Para problemas complejos a largo plazo"
    },
    "Model-Based RL": {
        "Descripción": "Aprender modelos del entorno para mejor planning",
        "Métodos": ["Dreamer", "PlaNet", "Latent-Space Models"],
        "Relevancia": "Mejor sample efficiency"
    },
    "Multi-Agent RL": {
        "Descripción": "Múltiples agentes compitiendo o cooperando",
        "Métodos": ["QMIX", "MADDPG", "CommNet"],
        "Relevancia": "Sistemas complejos y distribuidos"
    },
    "Vision + RL": {
        "Descripción": "Aprendizaje de políticas a partir de imágenes",
        "Métodos": ["DrQ", "SLAC", "Data Augmentation"],
        "Relevancia": "Problemas de visión real"
    }
}

for topic, info in future_topics.items():
    print(f"\n{topic}")
    print("-" * 80)
    print(f"  Descripción: {info['Descripción']}")
    print(f"  Métodos: {', '.join(info['Métodos'])}")
    print(f"  Relevancia: {info['Relevancia']}")

### 8.3 Guía de Implementación: Próximos Pasos

In [ ]:
print("RUTA DE APRENDIZAJE RECOMENDADA")
print("="*80)

ruta = {
    "Fase 1: Fundamentos (1-2 semanas)": {
        "Objetivo": "Entender conceptos básicos",
        "Actividades": [
            "Implementa PPO desde cero",
            "Experimenta con diferentes entornos",
            "Entiende GAE y clipping",
            "Visualiza training dynamics"
        ],
        "Recursos": "Notebooks 01-05 del repositorio"
    },
    "Fase 2: Algoritmos Avanzados (2-3 semanas)": {
        "Objetivo": "Dominar SOTA algorithms",
        "Actividades": [
            "Estudia DDPG, TD3, SAC en profundidad",
            "Implementa modificaciones (tuning, arquitecturas)",
            "Compara rendimiento sistemáticamente",
            "Entiende trade-offs"
        ],
        "Recursos": "Este notebook + papers originales"
    },
    "Fase 3: Aplicaciones (3-4 semanas)": {
        "Objetivo": "Aplicar a problemas reales",
        "Actividades": [
            "Robótica simulada (MuJoCo, PyBullet)",
            "Entornos con imágenes (Atari, Control Suite)",
            "Fine-tuning y transfer learning",
            "Debugging y diagnostics"
        ],
        "Recursos": "Entornos de OpenAI + documentación"
    },
    "Fase 4: Meta-Learning (4-6 semanas)": {
        "Objetivo": "Algoritmos que aprenden a aprender",
        "Actividades": [
            "MAML para quick adaptation",
            "Few-shot learning en RL",
            "Multi-task RL",
            "Proyectos integradores"
        ],
        "Recursos": "Papers de meta-learning + implementaciones"
    }
}

for fase, info in ruta.items():
    print(f"\n{fase}")
    print("-" * 80)
    print(f"Objetivo: {info['Objetivo']}")
    print(f"Actividades:")
    for act in info['Actividades']:
        print(f"  - {act}")
    print(f"Recursos: {info['Recursos']}")

### 8.4 Referencias y Recursos Finales

In [ ]:
print("REFERENCIAS Y RECURSOS")
print("="*80)

resources = {
    "Papers Clave": [
        "[1] Schulman et al. (2017) - PPO: https://arxiv.org/abs/1707.06347",
        "[2] Lillicrap et al. (2016) - DDPG: https://arxiv.org/abs/1509.02971",
        "[3] Fujimoto et al. (2018) - TD3: https://arxiv.org/abs/1802.09477",
        "[4] Haarnoja et al. (2018) - SAC: https://arxiv.org/abs/1801.01290"
    ],
    "Librerías Recomendadas": [
        "OpenAI Gym / Gymnasium: entornos estándar",
        "PyTorch / TensorFlow: redes neuronales",
        "Stable-Baselines3: implementaciones de referencia",
        "RLlib: entrenamiento distribuido",
        "Acme: frameworks modulares"
    ],
    "Datasets y Benchmarks": [
        "Atari 2600: juegos clásicos",
        "MuJoCo: control continuo",
        "DeepMind Control Suite: robótica",
        "D4RL: offline RL benchmarks",
        "Procedural Generation: entornos infinitos"
    ],
    "Comunidad y Eventos": [
        "OpenAI Spinning Up: cursos libres",
        "ICML, ICLR, NeurIPS: conferencias principales",
        "RL Discord Communities: comunidad activa",
        "Hugging Face Hub: modelos pre-entrenados",
        "Kaggle Competitions: desafíos prácticos"
    ]
}

for category, items in resources.items():
    print(f"\n{category}")
    print("-" * 80)
    for item in items:
        print(f"  {item}")

print("\n" + "="*80)
print("\n¡Felicitaciones! Has completado el tutorial de Algoritmos SOTA en Deep RL.")
print("\nPróximos pasos:")
print("  1. Implementa tus propias versiones desde cero")
print("  2. Experimenta con nuevos entornos")
print("  3. Lee los papers originales")
print("  4. Contribuye a la comunidad open-source")
print("  5. ¡Sigue aprendiendo y explorando!")
print("\n" + "="*80)